In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import geopy
from geopy.distance import geodesic
from shapely.geometry import Point

In [ ]:
distritos = gpd.read_file("./SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.shp")
populacao = pd.read_csv("./Agregados_por_CEP_CNEFE_Censo_2022/Agregados_por_CEP_CNEFE_Censo_2022.csv", sep = ";" )
estacoes = pd.read_parquet("estacoes_reestruturadas.parquet")
declividade = gpd.read_file("./declividade/sad6996_declividade.shp")

In [ ]:
def criar_circulos_estacao(data, radius_km):
    circles = []
    for _, row in data.iterrows():
        circle_points = []
        center = (row['lat'], row['lon'])
        for angle in range(0,360, 1): ## intervalo de 1 grau para cada ponto do círculo
            destination = geopy.distance.distance(kilometers = radius_km).destination(center, angle)
            circle_points.append(Point(destination.longitude, destination.latitude))
        circle = gpd.GeoSeries(circle_points).unary_union.convex_hull
        circles.append(circle)
    data.rename(columns = {'geometry':'point'}, inplace = True)
    circles_gdf = gpd.GeoDataFrame(data, geometry = circles)
    circles_gdf.rename(columns = {'geometry':'circle'}, inplace = True)
    return circles_gdf

def adicionar_estacoes_com_circulos_mapa(mapa, df_estacoes, radius_km):
    layer_estacoes =  folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    if "circle" not in df_estacoes:
        data = criar_circulos_estacao(df_estacoes, radius_km)
    else:
        data = df_estacoes.copy()
    data.set_geometry("circle", inplace = True)
    for _, row in data.iterrows():
        folium.CircleMarker(
                    location=[row['lat'],row['lon']],
                    color = "blue",
                    radius= 5,
                    tooltip= row['name'],
                    fill=True,
                    fill_opacity=1,
                    fill_color="blue",
                ).add_to(layer_estacoes)
    for _, row in data.iterrows():
        # cor = get_color(row['CLASSE'])
    
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        layer_estacoes.add_to(mapa)
        layer_circulos_estacoes.add_to(mapa)
        folium.LayerControl().add_to(mapa)
    return mapa

def get_color(valor):
    if valor == 1:
        return "yellow"
    elif valor == 2:
        return "orange"
    elif valor == 3:
        return "red"
    elif valor == 4:
        return "darkred"
    else:
        return "black" # error

def criar_mapa_sp():
    mapa_sp = folium.Map(location = [-23.550520, -46.633308], zoom_start = 12)
    return mapa_sp
    
def tratar_df_estacoes(df_estacoes):
    df_estacoes = gpd.GeoDataFrame(df_estacoes, geometry = gpd.points_from_xy(df_estacoes['lon'], df_estacoes['lat']))
    df_estacoes = df_estacoes.set_crs("EPSG:4326")
    return df_estacoes

def tratar_df_declividade(df_declividade):
    df_declividade = df_declividade.set_crs("EPSG:29183")  # Define o CRS, se ainda não estiver definido
    df_declividade = df_declividade.to_crs("EPSG:4326")   # Converte para WGS84 (latitude/longitude)
    substituicoes = {
        '0 a 05%': 1,
        '05 a 25%': 2,
        '25 a 60%': 3,
        '> 60%': 4
    }

    # Substituindo valores
    df_declividade['CLASSE'] = df_declividade['CLASSE'].replace(substituicoes).astype(int)
    return df_declividade
def analise_declividade(df_estacoes, df_declividade, radius_km):
    mapa = criar_mapa_sp()
    df_estacoes_circulo = criar_circulos_estacao(df_estacoes, radius_km)
    df_estacoes_circulo.set_geometry("circle", inplace=True)  
    cruzamentos =  gpd.sjoin(df_estacoes_circulo, df_declividade, how="inner", predicate="intersects")
    
    cruzamentos_finais = cruzamentos[[
        "name",  # Nome da estação
        "circle",  # Geometria 'circle' da estação
        "point",  # Geometria 'point' da estação
        "ID",     # ID da declividade
        "AREA",   # Área do distrito
        "CLASSE", # Código do distrito
    ]].copy()
    
    cruzamentos_finais["district_geometry"] = df_declividade.loc[cruzamentos["index_right"], "geometry"].values
    declividade_necessaria_para_plot = cruzamentos_finais.drop_duplicates(subset='district_geometry')

    declividade_necessaria_para_plot['intersection'] =  declividade_necessaria_para_plot.apply(lambda row: 
            (row['circle'].intersection(row['district_geometry'])).area if not row['circle'].intersection(row['district_geometry']).is_empty 
                                                                                               else 0, axis=1)
    
    # return declividade_necessaria_para_plot
    layer_declividade = folium.FeatureGroup(name = "Declividade")
    layer_estacoes =  folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    for _, row in declividade_necessaria_para_plot.iterrows():
        cor = get_color(row['CLASSE'])
        geo_json = row['district_geometry'].__geo_interface__
        folium.GeoJson(
            geo_json, 
            tooltip = str(row['ID']) + " Classe " + str(row['CLASSE']),
            style_function = lambda x, color = cor:{'fillColor' : color , 'color' : color, 'weight' : 1}
        ).add_to(layer_declividade)

    for _, row in df_estacoes_circulo.iterrows():
        folium.CircleMarker(
                    location=[row['lat'],row['lon']],
                    color = "blue",
                    radius= 5,
                    tooltip= row['name'],
                    fill=True,
                    fill_opacity=1,
                    fill_color="blue",
                ).add_to(layer_estacoes)
    for _, row in df_estacoes_circulo.iterrows():
        # cor = get_color(row['CLASSE'])
    
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        
    layer_declividade.add_to(mapa)
    layer_estacoes.add_to(mapa)
    layer_circulos_estacoes.add_to(mapa)
    folium.LayerControl().add_to(mapa)

    return mapa
    
    

In [ ]:
estacoes = tratar_df_estacoes(estacoes)
declividade = tratar_df_declividade(declividade)


In [ ]:
analise_declividade(estacoes,declividade, 1)

In [ ]:
populacao.head()

In [ ]:
estacoes.head()

In [ ]:
declividade.head()

In [ ]:
declividade.shape

In [ ]:
declividade.isnull().sum()

In [ ]:
declividade['CLASSE'].value_counts()